# Presplit/deterministic enrichment
The formated dataset still has some multi-entry columns (e.g. country). Creating a dummy for every country would lead to 171 extra columns (times the other multi-entry columns). This risks overfitting and a lot of missings. We thus group the countries in advance by their geographical location. Furthermore, SME and organizationID are multi-entry columns too. We create numerical columns to count the number of SMEs and organizations, respectively.

In [25]:
# Libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
from sklearn.pipeline     import Pipeline, FeatureUnion
from sklearn.impute       import SimpleImputer
from sklearn.compose          import TransformedTargetRegressor
from sklearn.preprocessing import OrdinalEncoder

In [11]:
df = pd.read_csv("eu_data.csv")

# Define geographic groups
geo_groups = {
    'Western Europe': {'DE', 'FR', 'BE', 'NL', 'LU', 'CH', 'AT', 'LI'},
    'Northern Europe': {'UK', 'IE', 'SE', 'FI', 'DK', 'IS', 'NO', 'EE', 'LV', 'LT'},
    'Southern Europe': {'IT', 'ES', 'PT', 'EL', 'MT', 'CY', 'SI'},
    'Eastern Europe': {'PL', 'CZ', 'SK', 'HU', 'RO', 'BG', 'RS', 'UA', 'AL', 'MK', 'ME', 'XK', 'HR', 'MD', 'GE', 'BA'},
    'Africa': {'ZA', 'KE', 'UG', 'TN', 'GH', 'MA', 'TZ', 'EG', 'SN', 'CD', 'MZ', 'RW', 'BF', 'ZM', 'CI', 'CM', 'ET', 'NG', 'DZ', 'AO', 'GN', 'BJ', 'GA', 'MW', 'ML', 'BI', 'MU', 'ST', 'LR', 'ZW', 'CG', 'GW', 'NE', 'LY', 'GQ', 'SD', 'LS', 'TD', 'DJ'},
    'Asia': {'IL', 'TR', 'IN', 'CN', 'JP', 'KR', 'TH', 'SG', 'LB', 'TW', 'UZ', 'AM', 'VN', 'MY', 'KZ', 'PK', 'AZ', 'HK', 'ID', 'JO', 'BD', 'KG', 'IR', 'PS', 'MN', 'KH', 'TJ', 'IQ', 'TM', 'NP', 'KW', 'QA', 'AF', 'BT', 'MO', 'MV', 'LA', 'LK'},
    'Oceania': {'AU', 'NZ', 'FJ', 'MH', 'PG', 'NC'},
    'Americas': {'US', 'CA', 'BR', 'AR', 'CO', 'CL', 'MX', 'PE', 'UY', 'BO', 'CR', 'PA', 'GT', 'SV', 'PY', 'EC', 'VE', 'DO', 'HT', 'SR', 'AW', 'BQ', 'AI', 'GU'}
}

# Number of partners from geo_groups 
for region, countries in geo_groups.items():
    df[f'{region}_count'] = (
        df['country']
          .str.split(';')  
          .apply(lambda lst: sum(c.strip() in countries for c in lst))
    )

# Number of countries per project
df['num_countries'] = df['country'].str.split(';').apply(lambda x: len([c.strip() for c in x if c.strip()]))

# Number of organizations per project (later choose only one of the 2 as feature because almost the same)
df['num_organisations'] = df['organisationID'].str.split(';').apply(lambda x: len([o.strip() for o in x if o.strip()]))

# Number of SMEs per project
def count_smes(s):
    entries = [e.strip() for e in s.split(';') if e.strip()]
    return sum(1 for e in entries if e.lower() == 'true')

df['num_sme'] = df['SME'].apply(count_smes)

# Number of different roles per project
df['role_list'] = df['role'].str.split(';').apply(lambda L: [r.strip() for r in L])
df['n_participant'] = df['role_list'].apply(lambda L: L.count('participant'))
df['n_associatedPartner']= df['role_list'].apply(lambda L: L.count('associatedPartner'))
df['n_thirdParty']= df['role_list'].apply(lambda L: L.count('thirdParty'))


We also want to transform the date columns into the more meaningful variables: starting year, starting month and duration, signature lag days.

In [12]:
for col in ['startDate', 'endDate', 'ecSignatureDate']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    
df['duration_days']      = (df.endDate - df.startDate).dt.days
df['start_year']         = df.startDate.dt.year
df['start_month']        = df.startDate.dt.month

The EuroSciVoc table has additional information about the topic domain. We want to include that in our data and analysis. 

In [13]:
# Load the Euroscivox data
df2 = pd.read_csv("EuroSciVox.csv", usecols=['projectID','euroSciVocPath'])

# Extract the first “folder” from the path as topic domain (the thing between the first two slashes)
df2['euroSciVoxTopic'] = df2['euroSciVocPath'].str.extract(r'^/([^/]+)/?')

# Keep one row per projectID
df2 = (
    df2[['projectID','euroSciVoxTopic']]
      .drop_duplicates(subset='projectID')
)

# Left‐merge into main df
df = df.merge(df2, on='projectID', how='left')

df['euroSciVoxTopic'] = df['euroSciVoxTopic'].fillna('not available')

print(df.head())


   projectID        acronym  status  \
0  101159220      PvSeroRDT  SIGNED   
1  101096150       BIOBoost  SIGNED   
2  101093997  GlycanTrigger  SIGNED   
3  101126531   CHIKVAX_CHIM  SIGNED   
4  101113979      The Oater  CLOSED   

                                               title  startDate    endDate  \
0  A point-of-care serological rapid diagnostic t... 2025-02-01 2030-01-31   
1  Boosting innovation agencies for bioeconomy va... 2023-02-01 2025-01-31   
2  GLYCANS AS MASTER TRIGGERS OF HEALTH TO INTEST... 2023-01-01 2028-12-31   
3  Late-stage clinical development of Chikungunya... 2023-06-01 2028-11-30   
4  The Oater develops a compact machine for hyper... 2023-07-01 2023-12-31   

   ecMaxContribution ecSignatureDate                              masterCall  \
0         4062396.23      2024-12-09  HORIZON-JU-GH-EDCTP3-2023-02-two-stage   
1          500000.00      2022-11-25             HORIZON-EIE-2022-CONNECT-01   
2         6771571.00      2022-12-05           HORIZON-H

In [14]:
df.to_csv("df_eu.csv", index=False)

# Feature Enrichment and Model

In [16]:
# 1) train/test split
target = df['ecMaxContribution']
X_train, X_test, y_train, y_test = train_test_split(
    df, target, test_size=0.2, random_state=42
)

We have the organisationID column and also the net_ecContribution_list column. We want to include a feature for each organisation’s “past mean EC” using only contributions from earlier projects (shifting the cumulative sum/count so the current row’s own contribution is excluded). In addition, we need a lookup (org_dim) that the transformer uses at predict time.

In [17]:
# 2) precompute lookup table
# organization lookup
train_long = (
    X_train[['projectID','organisationID','ecContribution_list','startDate']]
    .copy() 
)

# normalize delimiters
train_long['clean_orgs'] = (
    train_long['organisationID']
      .astype(str)
      .str.strip('[]')
      .str.replace(r'[;,]', ';', regex=True)
)

train_long['clean_ecs'] = (
    train_long['ecContribution_list']
      .astype(str)
      .str.strip('[]')
      .str.replace(r'[;,]', ';', regex=True)
)

# split into lists
train_long['org_list'] = train_long['clean_orgs'].str.split(';').apply(lambda lst: [o.strip() for o in lst if o.strip()])
train_long['ec_list']  = train_long['clean_ecs'].str.split(';').apply(lambda lst: [float(x)       for x in lst if x.strip()])

# explode
train_long['pairs'] = train_long.apply(lambda r: list(zip(r['org_list'], r['ec_list'])), axis=1)
train_long = train_long.explode('pairs')
train_long[['organisationID','ecContribution']] = pd.DataFrame(train_long['pairs'].tolist(), index=train_long.index)

# compute each org’s historical mean (excluding current)
train_long = train_long.sort_values(['organisationID','startDate'])
train_long['cum_sum']   = train_long.groupby('organisationID')['ecContribution'].cumsum().shift(1).fillna(0)
train_long['cum_count'] = train_long.groupby('organisationID')['ecContribution'].cumcount()
train_long['past_mean'] = (train_long['cum_sum'] / train_long['cum_count'].replace(0,np.nan)).fillna(0)

# build org_dim
org_dim = (
    train_long
      .groupby('organisationID')['past_mean']
      .last()
      .reset_index()
      .rename(columns={'past_mean':'org_past_mean_ec'})
)

We have textual features such as the objective column. We use LDA for topic (objective) modeling

In [18]:
# 3) Build & fit LDA pipeline on training “objective” text

# define extra stop-words
extra_stops = {'project', 'new', 'study', 'research', 'based', 'use'}
stop_words = list(ENGLISH_STOP_WORDS.union(extra_stops))

# LDA pipeline
lda_pipeline = Pipeline([
    ('vect', CountVectorizer(
        stop_words=stop_words,
        min_df=5,
        ngram_range=(1,2)
    )),
    ('lda', LatentDirichletAllocation(
        n_components=10,
        max_iter=20,
        learning_method='online',
        random_state=42,
        n_jobs=-1
    )),
])

# fit on train objectives
lda_pipeline.fit(X_train['objective'])

# top words per topic
def display_topics(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        top_features = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print(f"Objective #{topic_idx + 1}: {' '.join(top_features)}")

# display topics
vectorizer = lda_pipeline.named_steps['vect']
lda_model  = lda_pipeline.named_steps['lda']
feature_names = vectorizer.get_feature_names_out()

display_topics(lda_model, feature_names, n_top_words=10)

# objective‐weight matrix
doc_obj_dist = lda_pipeline.transform(X_train['objective'])

# dataframe with generic names
obj_cols = [f"obj_{i+1}" for i in range(doc_obj_dist.shape[1])]
objective_df = pd.DataFrame(
    doc_obj_dist,
    columns=obj_cols,
    index=X_train.index
)

# descriptive names
new_names = {
    'obj_1':  'obj_advanced_energy_storage_materials',
    'obj_2':  'obj_researcher_academic_training',
    'obj_3':  'obj_eu_climate_policy_data',
    'obj_4':  'obj_molecular_synthetic_biology',
    'obj_5':  'obj_clinical_cell_cancer_biology',
    'obj_6':  'obj_industrial_sustainable_energy',
    'obj_7':  'obj_global_environmental_change',
    'obj_8':  'obj_social_cultural_studies',
    'obj_9':  'obj_quantum_theoretical_physics',
    'obj_10': 'obj_digital_health_ai'
}

objective_df = objective_df.rename(columns=new_names)
train_df = X_train.join(objective_df)
print(X_train.head())

Objective #1: materials energy high devices low technology storage power co2 performance
Objective #2: training scientific researchers science skills european university knowledge academic expertise
Objective #3: data european eu climate support policy innovation solutions stakeholders digital
Objective #4: protein plant drug potential molecular novel chemical proteins develop synthetic
Objective #5: cell cells cancer disease patients clinical treatment brain tissue mechanisms
Objective #6: energy technology production market technologies industrial industry cost design sustainable
Objective #7: understanding role change global processes climate changes knowledge evolution environmental
Objective #8: social cultural political studies history heritage analysis practices gender historical
Objective #9: quantum systems time data models high physics methods field theory
Objective #10: data health ai learning human social risk care people life
      projectID              acronym  status  \

In [20]:
# 5a) wrap OrgAvgPastEC so that it _only_ returns its new column:
class OrgAvgPastEC(BaseEstimator, TransformerMixin):
    def __init__(self, org_dim):
        # org_dim: DataFrame(org, mean) or dict
        if hasattr(org_dim, 'set_index'):
            org_dim = org_dim.set_index('organisationID')['org_past_mean_ec'].to_dict()
        self.org_dim = org_dim

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X: DataFrame with at least projectID, organisationID, ecContribution_list
        proj_to_means = {}
        for _, row in X.iterrows():
            pid = row['projectID']
            orgs = (
                str(row['organisationID'])
                   .strip('[]')
                   .replace(r',', ';')
                   .split(';')
            )
            orgs = [o.strip() for o in orgs if o.strip()]
            means = [ self.org_dim.get(o, 0.0) for o in orgs ]
            proj_to_means[pid] = float(np.mean(means)) if means else 0.0

        arr = X['projectID'].map(proj_to_means).fillna(0.0).values
        return arr.reshape(-1,1)


# 5b) wrap ObjectiveMeanLookup similarly
class ObjectiveMeanEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, lda_pipeline):
        self.lda_pipeline = lda_pipeline

    def fit(self, X, y):
        doc_topic = self.lda_pipeline.transform(X['objective'])
        y_arr     = np.array(y).reshape(-1,1)
        sums      = (doc_topic * y_arr).sum(axis=0)
        weights   = doc_topic.sum(axis=0)
        self.topic_means_ = sums / weights
        return self

    def transform(self, X):
        doc_topic = self.lda_pipeline.transform(X['objective'])
        hist_mean = (doc_topic * self.topic_means_).sum(axis=1)
        return hist_mean.reshape(-1,1)

In [ ]:
# 5) Short exploration of numerical variables (distribution and boxplots)
numeric_cols = [
    'ecMaxContribution','duration_days','n_participant','n_associatedPartner',
    'n_thirdParty', 'Western Europe_count', 'Eastern Europe_count', 'Northern Europe_count',
    'Southern Europe_count', 'Africa_count', 'Asia_count', 'Oceania_count', 'Americas_count',
    'num_countries', 'num_organisations', 'num_sme', 'start_year']

for col in numeric_cols:
    plt.figure()
    plt.hist(X_train[col].dropna(), bins=30)
    plt.title(f"Distribution of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()

plt.figure(figsize=(12,6))
X_train[numeric_cols].boxplot(rot=45)
plt.title("Boxplots of Key Numeric Features")
plt.tight_layout()
plt.show()

In [31]:
# 6) Preprocess numerical and categorical features
# Custom target-transformer: winsorize at 1st/99th percentile, then log1p

class WinsorLogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower=0.01, upper=0.99):
        self.lower = lower
        self.upper = upper

    def fit(self, y, X=None):
        y = np.array(y).ravel()
        self._clip_lo, self._clip_hi = np.quantile(y, [self.lower, self.upper])
        return self

    def transform(self, y):
        y = np.array(y).ravel()
        y = np.clip(y, self._clip_lo, self._clip_hi)
        return np.log1p(y).reshape(-1, 1)

    def inverse_transform(self, y):
        return np.expm1(y)

target_transformer = WinsorLogTransformer(lower=0.01, upper=0.99)

# Cyclical encoder for month
class CyclicalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col, period):
        self.col = col
        self.period = period

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vals = X[self.col].astype(float).values
        sin = np.sin(2 * np.pi * vals / self.period)
        cos = np.cos(2 * np.pi * vals / self.period)
        return np.vstack([sin, cos]).T


month_encoder = Pipeline([
    ('cycle', CyclicalEncoder(col='start_month', period=12)),
    ('scale', StandardScaler()),
])

# numeric & categorical pipelines
num_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale',  StandardScaler()),
])

cat_cols = ['fundingScheme', 'masterCall', 'euroSciVoxTopic']
cat_pipe = Pipeline([
    ('impute', OrdinalEncoder()),  
])

# Put it all together in a FeatureUnion
preprocessor = FeatureUnion([
    ('numeric',      ColumnTransformer([('num', num_pipe, numeric_cols)], remainder='drop')),
    ('categorical',  ColumnTransformer([('cat', cat_pipe, cat_cols)], remainder='drop')),
    ('month_cycle',  month_encoder),
    ('topic_enc', topic_encoder),
    ('org_avg_ec',   Pipeline([
                         ('org_avg', OrgAvgPastEC(org_dim)),
                         ('scale',   StandardScaler())
                     ])),
    ('obj_mean_ec',  ObjectiveMeanEncoder(lda_pipeline)),
])


In [28]:
class SupervisedLDATopicEncoder(BaseEstimator, TransformerMixin):
    def __init__(self,
                 n_components=10,
                 vectorizer_params=None,
                 lda_params=None):
        self.n_components     = n_components
        self.vectorizer_params = vectorizer_params or {}
        self.lda_params        = lda_params or {}

    def fit(self, X, y):
        texts = X['objective']

        # 1) Fit the vectorizer on this fold
        self.vectorizer_ = CountVectorizer(**self.vectorizer_params)
        dtm = self.vectorizer_.fit_transform(texts)

        # 2) Fit the LDA on that DTM
        self.lda_ = LatentDirichletAllocation(
            n_components=self.n_components,
            **self.lda_params
        ).fit(dtm)

        # 3) Compute fold‐specific topic_means
        doc_topic = self.lda_.transform(dtm)       # shape = (n_samples, n_topics)
        y_arr     = np.array(y).reshape(-1,1)
        sums      = (doc_topic * y_arr).sum(axis=0)
        weights   = doc_topic.sum(axis=0)
        self.topic_means_ = sums / weights

        return self

    def transform(self, X):
        dtm       = self.vectorizer_.transform(X['objective'])
        doc_topic = self.lda_.transform(dtm)
        # return the weighted sum per doc
        return (doc_topic * self.topic_means_).sum(axis=1).reshape(-1,1)


In [30]:
topic_encoder = SupervisedLDATopicEncoder(
    n_components=10,
    vectorizer_params={
      'stop_words': stop_words,
      'min_df':     5,
      'ngram_range': (1,2)
    },
    lda_params={
      'max_iter':       20,
      'learning_method': 'online',
      'random_state':    42,
      'n_jobs':          -1
    }
)

In [32]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV  # or RandomizedSearchCV
import numpy as np

# 7) Train model
base_pipeline = Pipeline([
    ('features', preprocessor),
    ('model',    XGBRegressor(
        objective='reg:squarederror',
        random_state=42,
        verbosity=1,          
        tree_method='hist'    
    )),
])

pipeline = TransformedTargetRegressor(
    regressor=base_pipeline,
    transformer=target_transformer
)

# Parameter grid
param_grid = {
    'regressor__model__n_estimators':    [100, 300, 600],
    'regressor__model__max_depth':       [3, 6, 10],
    'regressor__model__learning_rate':   [0.01, 0.1, 0.2],
    'regressor__model__subsample':       [0.7, 1.0],
    'regressor__model__colsample_bytree': [0.6, 0.8, 1.0],
}

# search
search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=2,           
    return_train_score=True
)

# Fit the search on train data
search.fit(X_train, y_train)

print("Best R² (CV):", search.best_score_)
print("Best params:", search.best_params_)

# Evaluate
test_r2 = search.score(X_test, y_test)
print("Test R² with best model:", test_r2)


Fitting 5 folds for each of 162 candidates, totalling 810 fits


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.
